In [2]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import os
import skimage.io
from PIL import Image
import pandas as pd

In [3]:
%pwd

'/Users/jones/Library/CloudStorage/OneDrive-NorthwesternUniversity/ownProject/code/imagePreProcessing'

In [24]:
#macpath =
macbookpath = '/Users/jones/Library/CloudStorage/OneDrive-NorthwesternUniversity/ownProject/data'
imagesMacbookPath = '/Users/jones/Library/CloudStorage/OneDrive-NorthwesternUniversity/ownProject/data/rawData/clampFISCHIan/Subregion_44_r5_c4'
maskMacbookPath = '/Users/jones/Library/CloudStorage/OneDrive-NorthwesternUniversity/ownProject/data/segmentations/npyFiles'

In [5]:
os.chdir(macbookpath)

In [7]:
genes = pd.read_csv('spots.csv')

In [8]:
genes.head(5)

,spotID,x,y,intensity,nearestNucID,status,maskID,channel,distanceToNuc
0,1,921,772,24.67889,127,0,0,R1_YFP_UBC,149.96670
1,2,639,381,24.67972,65,0,0,R1_YFP_UBC,34.52535
2,3,380,834,24.68205,194,0,0,R1_YFP_UBC,108.90820
3,4,461,912,24.68855,188,0,0,R1_YFP_UBC,36.22154
4,5,953,106,24.68930,37,0,0,R1_YFP_UBC,384.29280


In [9]:
os.chdir(maskMacbookPath)
files = [f for f in listdir(maskMacbookPath) if isfile(join(maskMacbookPath, f))]
seg = np.load(files[0], allow_pickle= True).item()
img = seg['img']
masks = seg['masks']
len(np.unique(masks))

2464

In [10]:
len(genes)

629167

In [21]:
def maskCellpose(row, masks):
    return masks[(row['x']-1), (row['y']-1)]

In [13]:
genes['maskID'] = masks[genes['x'], genes['y']]

IndexError: index 7187 is out of bounds for axis 0 with size 7187

In [22]:
genes['maskCellpose'] = genes.apply (lambda row: maskCellpose(row, masks), axis=1)

In [23]:
genes.head(5)

,spotID,x,y,intensity,nearestNucID,status,maskID,channel,distanceToNuc,maskCellpose
0,1,921,772,24.67889,127,0,0,R1_YFP_UBC,149.96670,184
1,2,639,381,24.67972,65,0,0,R1_YFP_UBC,34.52535,129
2,3,380,834,24.68205,194,0,0,R1_YFP_UBC,108.90820,0
3,4,461,912,24.68855,188,0,0,R1_YFP_UBC,36.22154,101
4,5,953,106,24.68930,37,0,0,R1_YFP_UBC,384.29280,0


(7187, 7146)

In [30]:
thresholds = pd.read_csv(os.path.join(macbookpath, 'GenesThresholds.csv'))
thresholds

,Gene,Threshold
0,R1_YFP_UBC,37
1,R1_CY3_WNT5A,90
2,R1_A594_DDX58,30
3,R1_CY5_AXL,60
4,R2_YFP_UBC,40
5,R2_CY3_NGFR,45
6,R2_A594_FN1,55
7,R2_CY5_EGFR,45
8,R3_YFP_UBC,40
9,R3_CY3_ITGA3,45


In [ ]:
def intensityThreshold(row):
    if row['channel'] == 'R1_YFP_UBC':
        if row['intensity'] > 35:
            return 1
        else:
            return 0
    elif row['channel'] == 'R1_YFP_UBC':
        if row['intensity'] > 35:
            return 1
        else:
            return 0

In [66]:
def intensityThreshold(row):
    threshold = thresholds['Threshold'].loc[thresholds['Gene'] == row['channel']].iloc[0]
    if row['intensity'] > threshold:
        return 1
    else:
        return 0

In [67]:
genes['aboveThreshold'] = genes.apply (lambda row: intensityThreshold(row), axis=1)

In [68]:
genes.head(5)

,spotID,x,y,intensity,nearestNucID,status,maskID,channel,distanceToNuc,maskCellpose,aboveThreshold
0,1,921,772,24.67889,127,0,0,R1_YFP_UBC,149.96670,184,0
1,2,639,381,24.67972,65,0,0,R1_YFP_UBC,34.52535,129,0
2,3,380,834,24.68205,194,0,0,R1_YFP_UBC,108.90820,0,0
3,4,461,912,24.68855,188,0,0,R1_YFP_UBC,36.22154,101,0
4,5,953,106,24.68930,37,0,0,R1_YFP_UBC,384.29280,0,0
